# Requirement:
- numpy 1.19.5
- tensorflow 2.5.0
- tensorflow_privacy 0.6.1
- sklearn 0.24.2

Before performing the experiments, you need to fit the **pickle_file**(path for tabular QMNIST data) according to your own PC.

In [1]:
import numpy as np

import pickle
import tensorflow as tf
from tensorflow.keras import layers
import random as python_random

from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import euclidean_distances

# Load QMNIST data.

In [37]:
pickle_file = '/home/jiangnan/Desktop/ppml-workshop/data/QMNIST_tabular_ppml.pickle'

with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  x_defender = pickle_data['x_defender']
  x_reserve = pickle_data['x_reserve']
  y_defender = pickle_data['y_defender']
  y_reserve = pickle_data['y_reserve']
  del pickle_data
print('Data loaded.')

Data loaded.


In [38]:
NUM_CLASSES = 10

y_defender = y_defender[:,0]
y_reserve = y_reserve[:,0]

#y_defender = np.expand_dims(y_defender,axis=1)
#y_reserve = np.expand_dims(y_reserve,axis=1)

#y_defender = tf.keras.utils.to_categorical(y_defender, num_classes=NUM_CLASSES)
#y_reserve = tf.keras.utils.to_categorical(y_reserve, num_classes=NUM_CLASSES)

# Defender model $M_D$

import the defender model which need to be tested.

In [57]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KDTree
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

def defender_model_fn():
    """The architecture of the defender (victim) model.
    The attack is white-box, hence the attacker is assumed to know this architecture too."""
    
    #random_seeds = random.sample(range(0,100), 2)
    #np.random.seed(random_seeds[0])
    #python_random.seed(random_seeds[1])
    
    #model = LogisticRegression(random_state=None)
    #model = RidgeClassifier(random_state=None)
    #model = LinearSVC(random_state=None)
    #model = DecisionTreeClassifier(random_state=None)
    #model = KNeighborsClassifier() # no random_state
    #model = KDTree()
    #model = GaussianNB() # no random_state
    model = SVC(probability=True,random_state=None)
    #model = SGDClassifier(random_state=None)
    #model = Perceptron(random_state=None)
    #model = RandomForestClassifier(random_state=None)
    #model = MLPClassifier(random_state=None)
    
    return model

# if random_select, randomly extract two points from both defender & reserve dataset, and repeat for n_extract times
random_select = True
n_extract = 100

#if given_index, the attack will put the two left out points back to their original place
given_index = False

# Oracle attack model $M_A$

In [58]:
import random
from tqdm import tqdm

In [59]:
def create_mock_defender_models(defender, data_in, data_out, n_records = 48, random_select = True, n_extract = 100, given_index = False):
    
    difference_in = []
    difference_out = []
    
    number_loop = 0
    
    if random_select == True:
        number_loop = n_extract
    else:
        number_loop = data_in[0].shape[0]
        
    for i in tqdm(range(number_loop)):

        if random_select == True:
            index = random.randint(0,n_records-1)
        else:
            index = i

        evaluation_data_in = data_in[0][index]
        evaluation_label_in = data_in[1][index]

        evaluation_data_out = data_out[0][index]
        evaluation_label_out = data_out[1][index]

        evaluation_data = np.array([evaluation_data_in, evaluation_data_out])
        evaluation_label = np.array([evaluation_label_in, evaluation_label_out])

        evaluation = evaluation_data, evaluation_label


        attack_train_data_in = np.delete(data_in[0], index, axis=0)
        attack_train_label_in = np.delete(data_in[1], index, axis=0)

        attack_in = attack_train_data_in, attack_train_label_in


        attack_train_data_out = np.delete(data_out[0], index, axis=0)
        attack_train_label_out = np.delete(data_out[1], index, axis=0)

        attack_out = attack_train_data_out, attack_train_label_out

        #predict = defender_model.predict(attack_in[0])
        predict = defender_model.predict_proba(attack_in[0])
        
        if given_index == True:

            attack_in_plus_one_in = np.insert(attack_in[0], index, evaluation[0][0].reshape(1,attack_in[0].shape[1]), axis=0), np.insert(attack_in[1], index, evaluation[1][0], axis=0)
            attack_in_plus_one_out = np.insert(attack_in[0], index, evaluation[0][1].reshape(1,attack_in[0].shape[1]), axis=0), np.insert(attack_in[1], index, evaluation[1][1], axis=0)

        else:

            attack_in_plus_one_in = np.vstack((evaluation[0][0].reshape(1,attack_in[0].shape[1]),attack_in[0])), np.hstack(( evaluation[1][0],attack_in[1]))
            attack_in_plus_one_out = np.vstack((evaluation[0][1].reshape(1,attack_in[0].shape[1]),attack_in[0])), np.hstack(( evaluation[1][1],attack_in[1]))

        
        M_cD_in = defender_model_fn()
        M_cD_out = defender_model_fn()

        M_cD_in.fit(attack_in_plus_one_in[0], attack_in_plus_one_in[1])
        M_cD_out.fit(attack_in_plus_one_out[0], attack_in_plus_one_out[1])
        
        #M_cD_in_predict = M_cD_in.predict(attack_in[0])
        #M_cD_out_predict = M_cD_out.predict(attack_in[0])
        
        M_cD_in_predict = M_cD_in.predict_proba(attack_in[0])
        M_cD_out_predict = M_cD_out.predict_proba(attack_in[0])

        diff_in = np.mean(np.linalg.norm(M_cD_in_predict-predict))
        diff_out = np.mean(np.linalg.norm(M_cD_out_predict-predict))

        difference_in.append(diff_in)
        difference_out.append(diff_out)
    
    return difference_in, difference_out


# Start experiments:

In [60]:
accuracy_in_all = []
accuracy_out_all = []

difference_in_all = []
difference_out_all = []

n_records = 1600
n_trials = 3

for i in range(n_trials):
    
    random_indexes = random.sample(range(0,200000), n_records)
    data_in = x_defender[random_indexes], y_defender[random_indexes]
    data_out = x_reserve[random_indexes], y_reserve[random_indexes]
    
    
    defender_model = defender_model_fn()
    defender_model.fit(data_in[0],data_in[1])
    
    predict_in_proba = defender_model.predict(data_in[0])
    #predict_in = np.argmax(predict_in_proba, axis=1)
    acc_in = accuracy_score(data_in[1], predict_in_proba)
    
    
    predict_out_proba = defender_model.predict(data_out[0])
    #predict_out = np.argmax(predict_out_proba, axis=1)
    acc_out = accuracy_score(data_out[1], predict_out_proba)
    
    '''
    auc_by_class = []
    # compute auc per class then take the average value
    for i in range(NUM_CLASSES):
      class_indices = data_out[1] == i
      if np.sum(class_indices) == 0:
        continue
      fpr, tpr, thresholds = metrics.roc_curve(class_indices, predict_reserve_proba[:,i])
      auc = metrics.auc(fpr, tpr)
      auc_by_class.append(auc)
    
    average_auc = np.mean(auc_by_class)
    utility = max(2*average_auc -1,0)
    
    defender_acc_all.append(acc)
    defender_auc_all.append(average_auc)
    '''
    
    accuracy_in_all.append(acc_in)
    accuracy_out_all.append(acc_out)
    
    
    difference_in, difference_out = create_mock_defender_models(defender = defender_model,
                                                                    data_in = data_in,
                                                                    data_out = data_out,
                                                                    n_records = n_records,
                                                                    random_select = random_select,
                                                                    n_extract = n_extract,
                                                                    given_index = given_index)
    
    difference_in_all.append(difference_in)
    difference_out_all.append(difference_out)

100%|██████████| 100/100 [07:09<00:00,  4.30s/it]


In [61]:
accuracy_out_all

[0.914375, 0.909375, 0.91875]

In [53]:
print(np.mean(accuracy_in_all))
print(np.mean(accuracy_out_all))

0.9385416666666667
0.9187500000000001


In [62]:
# compute the privacy values by comparing all model pairs

privacy_all = []
variance_all = []
sigma_error_all = []

for i in range(len(difference_in_all)):
    
    difference_in = difference_in_all[i]
    difference_out = difference_out_all[i]
    
    results = []

    for j in tqdm(range(len(difference_in))):
        for k in range(len(difference_out)):

            if difference_in[j] <= difference_out[k]:
                results.append(1)
            else:
                results.append(0)

    n = len(results)
    p = 1-np.sum(results)/n

    privacy = min(2*p,1)
    variance = 2*p*(1-p)/n
    sigma_error = np.sqrt(p*(1-p)/2*n)
    
    privacy_all.append(privacy)
    variance_all.append(variance)
    sigma_error_all.append(sigma_error)
    

100%|██████████| 100/100 [00:00<00:00, 22223.83it/s]


In [63]:
print(np.mean(privacy_all))
print(np.mean(variance_all))
print(np.mean(sigma_error_all))

0.7530666666666667
4.6494383999999996e-05
34.0851958338298


In [56]:
privacy_all

[0.09499999999999997, 0.0, 0.21419999999999995]